# Web Scraping Facebook with Selenium
### Download ChromeDriver
You can either download latest stable release of ChromeDriver from:
https://chromedriver.chromium.org/
<br>
### OR Use webdriver_manager Istead
Or, you can install a library named webdriver_manager that will install it for you.
<br>
https://pypi.org/project/webdriver-manager/
<br>
<br>
Thank you so much to <b>MariyaSha</b>
https://github.com/MariyaSha/WebscrapingFacebook
<br>


### Mount Google Drive

In [1]:
from google.colab import drive

# Mount Google Drive at /content/drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/FacebookScraper

/content/drive/MyDrive/FacebookScraper


### Setting up Google Colab

In [3]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.atanV1aQ0A/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.dwnN4Jh7P4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.HmH0uIbW7q/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [31]:
%%shell
# Install chromium and chromium-driver
apt-get -q update
apt-get install -q chromium chromium-driver

# Install selenium
pip install -q selenium
pip install -q -U webdriver-manager
pip install -q tqdm
pip install -q webdriver_manager
pip install -q wget

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 110 kB in 1s (77.2 kB/s)
Reading package lists...
Reading packag

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from urllib.parse import urljoin
from IPython.display import clear_output
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ChromeOptions

## Step 1: Disable Alerts/Notifications

!! Using code by <b>pythonjar</b> of <b>Stackoverflow</b> !!
<br>
Please visit this thread for more details:
<br>
https://stackoverflow.com/questions/41400934/handle-notifications-in-python-selenium-chrome-webdriver

In [18]:
# Specify the path to chromedriver.exe (download and save on your computer)
service = Service(executable_path='/usr/bin/chromedriver')
options = ChromeOptions()
options.binary_location = "/usr/bin/chromium"
# Add any desired options to chrome_options here
options.add_argument('--disable-notifications')
options.add_argument('--incognito')
options.add_argument('--start-maximized')
# browser with no UI
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [19]:
# a public page
# profile = "https://www.facebook.com/deutschmitsajin"
# a public user
profile = "https://www.facebook.com/courtoxuar3"
profile = urljoin(profile+"/", "photos")
profile

'https://www.facebook.com/courtoxuar3/photos'

In [20]:
# getting first photo
driver = webdriver.Chrome(service=service, options=options)
driver.get(profile)
user = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//h1")))[0].get_attribute("innerText")
a_tags = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href,'https://www.facebook.com/photo.php?fbid')]")))
url = a_tags[0]
url = url.get_attribute("href")
driver.quit()
url

'https://www.facebook.com/photo.php?fbid=174828815626857&set=pb.100092992179841.-2207520000&type=3'

In [21]:
# Initialize a list to store images
images = []

In [22]:
# a public page
# url = "https://www.facebook.com/photo.php?fbid=269694472550009&set=pb.100085283753281.-2207520000&type=3"
# a public user
# url = "https://www.facebook.com/photo.php?fbid=174828815626857&set=pb.100092992179841.-2207520000&type=3"
url = url.replace(".php", "")
url = url.replace("&type=3", "")
# Create a Chrome WebDriver with the desired options
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
index = 0
while True:
    try:
        # Wait for the presence of all img elements
        img_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "img[data-visualcompletion='media-vc-image']")))
        image = img_elements[-1].get_attribute("src")
        if image and "https://scontent" in image and image not in images:
            index+=1
            images.append(image)
        # previous photo button
        # el = driver.find_elements(By.CSS_SELECTOR, "i[data-visualcompletion='css-img']")[1]
        # next photo button
        el = driver.find_elements(By.CSS_SELECTOR, "i[data-visualcompletion='css-img']")[2]
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(el)).click()
    except Exception as e:
        continue
    finally:
        clear_output(wait=True)
        print(len(images))
        if driver.current_url == url:
            break

driver.quit()

53


In [25]:
# with open('images_list.txt', 'w') as file:
#     for i in images:
#         file.write(f"{i}\n")

### Step 4: Create a new directory in the current folder


using OS path

In [28]:
path = os.getcwd()
path = os.path.join(path, user)

#create the directory
if not os.path.exists(path):
    os.mkdir(path)

In [29]:
# import shutil
# if os.path.exists(path):
#     shutil.rmtree(path)

### Step 5: Save all the photos in the new directory

Using wget

In [32]:
import wget
from tqdm import tqdm  # Import tqdm correctly

counter = 1
for image in tqdm(images, desc="Downloading..."):
    save_as = os.path.join(path, str(counter) + '.jpg')
    wget.download(image, save_as)
    counter += 1


Downloading...: 100%|██████████| 53/53 [00:01<00:00, 28.69it/s]
